# Libraries + function declaration

In [142]:
# For data preprocessing, aggregation.
import xmltodict # XML parser.
import json
import pandas as pd # To manage dataframes.
from bs4 import BeautifulSoup # HTML parser used to strip HTML from Posts.
import math

# For data scraping.
from requests.sessions import Session
import requests
import scrape # Scaper from Wikipedia.

def xml_to_data_frame(file_path: str) -> pd.DataFrame:
    with open(file_path, 'r', encoding="utf8") as f:
        dict = xmltodict.parse(f.read())
    
    key = list(dict)[0] # Get variable key.
    data = dict[key]['row'] # Used to access the data itself.
    df = pd.DataFrame.from_dict(data)

    print(f"\"{key}\" file successfully read and converted to dataframe.\n" + 
        f"Number of rows: {df.shape[0]}\n************************\n")
    
    return df

def strip_html(text: str):
    if isinstance(text, float):
        return text
        
    return BeautifulSoup(text, "lxml").text

def process_tags(tag):
    """ Tag format:
        * <tag 1><tag 2>...<tag N> IF N tags (str type)
        * nan IF 0 tags (float type)

        Returns empty list if there are no tags
        and a list of tags otherwise.
    """
    tags = []
    if isinstance(tag, float):
        return tags
    
    # Find open and closing bracket, add tag to list
    # <database-theory><relational-algebra>
    for t in tag.split(">"):
        tags.append(t[1:])

    if "" in tags:
        tags.remove("") # to remove the last extra "" added

    return tags

    

# Manual data cleaning & aggreation
Each dataset has to be manually processed.


## 1/5 - Comments

In [121]:
df_comments = xml_to_data_frame('Datasets/Comments.xml')

"comments" file successfully read and converted to dataframe.
Number of rows: 174347
************************



In [122]:
df_comments.columns

Index(['@Id', '@PostId', '@Score', '@Text', '@CreationDate', '@UserId',
       '@ContentLicense', '@UserDisplayName'],
      dtype='object')

In [123]:
# We only want to keep Id, PostId, Text and UserId
df_comments = df_comments.reindex(columns=["@Id", "@PostId", "@Text","@UserId"])

# Renaming of the columns
df_comments = df_comments.rename(columns={"@Id": "id", "@PostId": "postId", "@Text": "text", "@UserId": "userId"})

df_comments.head(1)

,id,postId,text,userId
0,8,2,To show that they have the same expressive pow...,10


In [124]:
# Finally, write into json
df_json = df_comments.to_json(orient="records")
parsed = json.loads(df_json)

with open("Datasets/comments.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)
    print("Comments dataset successfully converted to JSON!")

Comments dataset succesfully converted to JSON!


---
## 2/5 - Posts

In [147]:
df_posts = xml_to_data_frame('Datasets/Posts.xml')

"posts" file successfully read and converted to dataframe.
Number of rows: 92896
************************



In [148]:
df_posts.columns

Index(['@Id', '@PostTypeId', '@AcceptedAnswerId', '@CreationDate', '@Score',
       '@ViewCount', '@Body', '@OwnerUserId', '@LastEditorUserId',
       '@LastEditDate', '@LastActivityDate', '@Title', '@Tags', '@AnswerCount',
       '@CommentCount', '@FavoriteCount', '@ContentLicense', '@ParentId',
       '@OwnerDisplayName', '@ClosedDate', '@LastEditorDisplayName',
       '@CommunityOwnedDate'],
      dtype='object')

In [149]:
""" We want to keep Id, PostTypeId, AcceptedAnswerId, CreationDate, Score,
        ViewCount, Body, OwnerUserId, Title, Tags, ParentId
"""
df_posts = df_posts.reindex(columns=['@Id', '@PostTypeId', '@AcceptedAnswerId', '@CreationDate', '@Score',
       '@ViewCount', '@Body', '@OwnerUserId', '@Title', '@Tags','@ParentId'])

# Renaming of the columns
df_posts = df_posts.rename(columns={"@Id": "id", "@PostTypeId": "postTypeId",
        '@AcceptedAnswerId': 'acceptedAnswerId', '@CreationDate': 'creationDate',
        '@Score': 'score', '@ViewCount': 'viewCount','@Body': 'body', '@OwnerUserId':'ownerUserId',
        '@Title':'title', '@Tags':'tags','@ParentId':'parentId'})


# Strip the HTML formating from title and body:
df_posts['title'] = df_posts['title'].apply(lambda x: strip_html(x))
df_posts['body'] = df_posts['body'].apply(lambda x: strip_html(x))



df_posts.head(1)

,id,postTypeId,acceptedAnswerId,creationDate,score,viewCount,body,ownerUserId,title,tags,parentId
0,2,1,28,2012-03-06T19:06:05.667,19,1128,"The set difference operator (e.g., EXCEPT in s...",5,Does the 'difference' operation add expressive...,<database-theory><relational-algebra><finite-m...,NaN


In [150]:
# Tags should be in list format
# type(df_posts['tags'][3]) # nan type -> float
# type(df_posts['tags'])[0] # tags type -> str

df_posts['tags'] = df_posts['tags'].apply(lambda x: process_tags(x))

df_posts.head(1)

,id,postTypeId,acceptedAnswerId,creationDate,score,viewCount,body,ownerUserId,title,tags,parentId
0,2,1,28,2012-03-06T19:06:05.667,19,1128,"The set difference operator (e.g., EXCEPT in s...",5,Does the 'difference' operation add expressive...,"[database-theory, relational-algebra, finite-m...",NaN


In [151]:
# Finally, write into json
df_json = df_posts.to_json(orient="records")
parsed = json.loads(df_json)

with open("Datasets/posts.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)
    print("Posts dataset successfully converted to JSON!")

Posts dataset successfully converted to JSON!


---
## 3/5 - Tags

In [143]:
df_tags = xml_to_data_frame('Datasets/Tags.xml')

"tags" file successfully read and converted to dataframe.
Number of rows: 637
************************



In [144]:
df_tags.columns

Index(['@Id', '@TagName', '@Count', '@ExcerptPostId', '@WikiPostId'], dtype='object')

In [145]:
# We only want to keep Id and TagName
df_tags = df_tags.reindex(columns=["@Id", "@TagName"])

# Renaming of the columns
df_tags = df_tags.rename(columns={"@Id": "id", "@TagName": "tagName"})

df_tags.head(1)

,id,tagName
0,1,cpu-pipelines


In [146]:
# Aggregate descriptions scraped from Wikipedia.
# For that purpose, the scrape.py script is used.


with requests.Session() as s:
    #d = scrape.get_description(df_tags['tagName'][0], s)
    df_tags['description'] = df_tags['tagName'].apply(scrape.get_description, session=s)


KeyboardInterrupt: 

In [117]:
df_tags.head(1)

,id,tagName,description
0,1,cpu-pipelines,"In computing, a pipeline, also known as a data..."


In [ ]:
# Finally, write into json
df_json = df_tags.to_json(orient="records")
parsed = json.loads(df_json)

with open("Datasets/posts.json", "w", encoding="utf-8") as f:
    json.dump(parsed, f, indent=4)
    print("Posts dataset successfully converted to JSON!")

---
## 4/5 - Users

---
## 5/5 - Votes

---
# Testing

In [17]:
# For testing purpouses.
# WORKS.
# 18.6 secons to execute.
def test_xml_reading():
    list_of_xml = [
    'Datasets/Comments.xml',
    'Datasets/Posts.xml',
    'Datasets/Tags.xml',
    'Datasets/Users.xml',
    'Datasets/Votes.xml'
    ]

    for file in list_of_xml:
        xml_to_data_frame(file)

test_xml_reading()

"comments" file successfully read and converted to dataframe.
Number of rows:174347
************************

"posts" file successfully read and converted to dataframe.
Number of rows:92896
************************

"tags" file successfully read and converted to dataframe.
Number of rows:637
************************

"users" file successfully read and converted to dataframe.
Number of rows:119115
************************

"votes" file successfully read and converted to dataframe.
Number of rows:399089
************************



In [85]:
# Dictionary as value in pandas.Dataframe
d = {'col1': [{'a':'a'},{'b':'b'}], 'col2': [3, 4]}
df = pd.DataFrame(data=d)

# Encontrar valor del tag en dataframe con .where!

,col1,col2
0,{'a': 'a'},3
1,{'b': 'b'},4
